In [1]:
### 필요한 모듈 IMPORT
import pandas as pd
from sklearn import preprocessing
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [3]:
# 파일 읽어오기
a=pd.read_csv(r"C:\Users\moonp\ASAC과정\ASAC_ml_project\web_crawling\merge_dataset\raw_merge.csv",sep="\t")
### METACRITIC 점수 표기
a.metacritic=a.metacritic.fillna(-1)
tmp=a.dropna(axis=0)
### 동접자수 전처리 과정 동접자수 없는 경우 0으로 표기
tmp["24_Hour_Peak"].replace("Invalid","0",inplace=True)
tmp["All_time_peak"].replace("Invalid","0",inplace=True)
tmp["24_Hour_Peak"]=tmp["24_Hour_Peak"].astype(int)
tmp["All_time_peak"]=tmp["All_time_peak"].astype(int)

### is free 전처리
tmp.is_free.replace("False","0",inplace=True)
tmp.is_free.replace("0.0","0",inplace=True)
tmp.is_free.replace("True","1",inplace=True)
tmp.is_free.replace("1.0","1",inplace=True)
tmp.is_free=tmp.is_free.astype(int)
tmp.is_free=tmp.is_free.astype(int)


### TARGET CLASS를 4개로 재분류
No_interest_lst=['No user reviews','8 user reviews','7 user reviews','4 user reviews','9 user reviews','5 user reviews','6 user reviews','2 user reviews','3 user reviews','1 user reviews','None']
Positive_lst=['Overwhelmingly Positive','Very Positive','Positive','Mostly Positive']
Negative_lst=['Negative','Very Negative','Overwhelmingly Negative','Mostly Negative']
tmp.Review.replace(No_interest_lst,"No_interest",inplace=True)
tmp.Review.replace(Positive_lst,"Positive",inplace=True)
tmp.Review.replace(Negative_lst,"Negative",inplace=True)

### TARGET LABEL ENCODING
le=preprocessing.LabelEncoder()
le.fit(np.array(tmp.Review))
y_true=le.fit_transform(tmp.Review)
print(le.classes_)

['Mixed' 'Negative' 'No_interest' 'Positive']


c:\Users\moonp\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\series.py:4563: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
<ipython-input-3-805a1b8fccdd>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp["24_Hour_Peak"]=tmp["24_Hour_Peak"].astype(int)
<ipython-input-3-805a1b8fccdd>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/